In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import pandas as pd
import requests
import time

CON CSV

In [ ]:
#Read csv and create DF

UFO_df = pd.read_csv("./RESOURCES/nuforc_mexico_reports_detailed.csv")

UFO_df.drop(columns=UFO_df.columns[0], inplace=True)

UFO_df

In [ ]:
# Delete spaces on headers (required for the next step, since there are spaces before/after titles)
# For example [' Explanation '] to ['Explanation']
UFO_df.rename(columns=lambda x: x.strip(), inplace=True)

#drop columns that don't give relevant information
UFO_df.drop(columns=['Explanation','Reported','Detailed Description','Media'], inplace=True)

In [ ]:
print(UFO_df.columns)

In [ ]:
#create a new column with the information that is in parenthesis in City

UFO_df['Notas'] = UFO_df['City'].str.extract(r'\((.*?)\)')

#delete the information in parenthesis in the column City

UFO_df['City']=UFO_df['City'].str.replace(r'\((.*?)\)','',regex=True)

UFO_df

In [ ]:
# Crear una condición que verifique si la ciudad contiene 'jalisco' o 'Jalisco'
city_jalisco = UFO_df['City'].str.contains(r'jalisco', case=False, na=False)
notas_jalisco = UFO_df['Notas'].str.contains(r'jalisco', case=False, na=False)

# Actualizar la columna 'state' con 'Jalisco' donde se cumple la condición
UFO_df.loc[city_jalisco, 'State'] = 'Jalisco'

UFO_df.loc[notas_jalisco, 'State'] = 'Jalisco'



UFO_df.loc[UFO_df['State'] == 'Jalisco']



SCRAPPING

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
# Visit the NUFORC REPORTS UFO SIGHTINGS site
url = 'https://nuforc.org/subndx/?id=cMexico'
browser.visit(url)

In [4]:
# Scrape the website
html = browser.html

# Create a BeautifulSoup object from the scraped HTML
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Initialize variables to store scraped data
all_data = []  # List to store table data
links = []     # List to store links
# Loop until the "Next" link is clicked 4 times
# The underscore means "iterate four times, and I don't care about the loop variable."
for _ in range(4):
    
    ''' EXTRACT DATA AND LINKS FROM THE TABLE INTO LISTS
     '#table_1 tr' is a CSS selector used to select all <tr> (table row) elements within
     the HTML table that has an id attribute equal to 'table_1'.'''
    
    table_rows = soup.select('#table_1 tr')[1:]  # # the [1:] means skip the first row (headers)
    for row in table_rows:
        # Extract text data from each <td> element in the row and removes leading and trailing whitespace characters from each extracted text
        row_data = [td.get_text(strip=True) for td in row.find_all('td')]
        all_data.append(row_data)  # Append row data to the list of all data
        # Extract links from <a> elements in the row
        row_links = [f"https://nuforc.org{a['href']}" for a in row.find_all('a', href=True)]
        links.extend(row_links)  # Extend the list of links with links from the current row
    # Find the "Next" link and click it
    next_link = browser.find_by_id('table_1_next')
    next_link.click()
    # Wait for the page to load
    time.sleep(2)  # Adjust the sleep time as needed
    # Update the HTML content after navigating to the next page
    html = browser.html
    # Create a new BeautifulSoup object from the updated HTML
    soup = BeautifulSoup(html, 'html.parser')
# Extract headers from the table
headers = [header.get_text(strip=True) for header in soup.select('#table_1 th')]
# Convert the list of lists into a pandas DataFrame
table_df = pd.DataFrame(all_data, columns=headers)  # Create DataFrame for table data
links_df = pd.DataFrame(links, columns=['Link'])    # Create DataFrame for links


     Link          Occurred                City                State Country  \
0    Open  05/15/2024 02:20            Mazatlán              Sinaloa  Mexico   
1  Open !  05/12/2024 20:58         Los Médanos      Baja California  Mexico   
2    Open  05/09/2024 21:38   San José del Cabo  Baja California Sur  Mexico   
3    Open  02/26/2024 19:40    Ciudad de México     Ciudad de México  Mexico   
4    Open  12/28/2023 20:49  Acapulco de Juárez             Guerrero  Mexico   

      Shape                                            Summary Media  \
0     Light  Lights dropping from very high altitude and th...         
1  Cylinder  We observed a very large (500-1500 ft long), d...         
2  Triangle  We saw a triangular/round bright light in the ...     Y   
3       Orb  I was walking down the street coming to my hom...     Y   
4    Circle                           Maybe drone or maybe ufo     Y   

  Explanation  
0              
1              
2      Rocket  
3              
4     

In [6]:
    # Display the table_df
table_df

,Link,Occurred,City,State,Country,Shape,Summary,Media,Explanation
0,Open,05/15/2024 02:20,Mazatlán,Sinaloa,Mexico,Light,Lights dropping from very high altitude and th...,,
1,Open !,05/12/2024 20:58,Los Médanos,Baja California,Mexico,Cylinder,"We observed a very large (500-1500 ft long), d...",,
2,Open,05/09/2024 21:38,San José del Cabo,Baja California Sur,Mexico,Triangle,We saw a triangular/round bright light in the ...,Y,Rocket
3,Open,02/26/2024 19:40,Ciudad de México,Ciudad de México,Mexico,Orb,I was walking down the street coming to my hom...,Y,
4,Open,12/28/2023 20:49,Acapulco de Juárez,Guerrero,Mexico,Circle,Maybe drone or maybe ufo,Y,Drone?
...,...,...,...,...,...,...,...,...,...
399,Open,02/16/1984 23:00,"Guadalajara, Jalisco (Mexico)",,Mexico,Formation,it happend while on a road to guadalajara jali...,,
400,Open,06/01/1983 20:30,Ciudad Lazaro Cardenas (Mexico),,Mexico,Sphere,We saw a gigantic sphere in the sky one Saturd...,,
401,Open,05/03/1983 22:00,Highway (Mexico),,Mexico,Oval,Huge light formation spanned across (and above...,,
402,Open,08/01/1982 13:00,Cozumel (Mexico),,Mexico,,Can someone give me an idea what this is?,,


In [7]:
    # Display the links_df
links_df.head()

,Link
0,https://nuforc.org/sighting/?id=181515
1,https://nuforc.org/sighting/?id=181495
2,https://nuforc.org/sighting/?id=181446
3,https://nuforc.org/sighting/?id=180712
4,https://nuforc.org/sighting/?id=179884


In [8]:
# Drop the first column in the table_df
table_df.drop(columns=table_df.columns[0], inplace=True)
# Merge both dataframes to include the links column (replacing the dropped one)
merged_df = pd.merge(table_df, links_df, left_index=True, right_index=True)
merged_df.head()

,Occurred,City,State,Country,Shape,Summary,Media,Explanation,Link
0,05/15/2024 02:20,Mazatlán,Sinaloa,Mexico,Light,Lights dropping from very high altitude and th...,,,https://nuforc.org/sighting/?id=181515
1,05/12/2024 20:58,Los Médanos,Baja California,Mexico,Cylinder,"We observed a very large (500-1500 ft long), d...",,,https://nuforc.org/sighting/?id=181495
2,05/09/2024 21:38,San José del Cabo,Baja California Sur,Mexico,Triangle,We saw a triangular/round bright light in the ...,Y,Rocket,https://nuforc.org/sighting/?id=181446
3,02/26/2024 19:40,Ciudad de México,Ciudad de México,Mexico,Orb,I was walking down the street coming to my hom...,Y,,https://nuforc.org/sighting/?id=180712
4,12/28/2023 20:49,Acapulco de Juárez,Guerrero,Mexico,Circle,Maybe drone or maybe ufo,Y,Drone?,https://nuforc.org/sighting/?id=179884


In [ ]:
base_url = "https://nuforc.org/subndx/?id=cMexico&page="
# Número de páginas a scraper
num_pages = 5  # Ajusta este valor según sea necesario
# Crear un DataFrame para almacenar los datos
df = pd.DataFrame()
# Iterar sobre las páginas
for page in range(1, num_pages + 1):
    # Construir la URL para la página actual
    url = base_url + str(page)
    # Enviar solicitud GET para la página actual
    response = requests.get(url)
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontrar la tabla que contiene los datos
    table = soup.find('table', {'class': 'table-striped'})
    # Extraer los encabezados de la tabla
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]
    # Extraer los datos de la tabla
    data = []
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append([col for col in cols])
    # Convertir los datos en un DataFrame de Pandas
    page_df = pd.DataFrame(data, columns=headers)
    # Agregar los datos de la página actual al DataFrame principal
    df = pd.concat([df, page_df], ignore_index=True)
# Mostrar el DataFrame completo
print(df.head())
